# Bufstream Iceberg quickstart

In [ ]:
from pyspark.sql import SparkSession

conf = (
    pyspark.SparkConf()
        .setAppName('Jupyter')
        .set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
        .set("spark.sql.catalog.bufstream-quickstart", "org.apache.iceberg.spark.SparkCatalog")
        .set("spark.sql.catalog.bufstream-quickstart.type", "rest")
        .set("spark.sql.catalog.bufstream-quickstart.uri", "http://iceberg-rest:8181")
)
spark = SparkSession.builder.config(conf=conf).getOrCreate()


In [ ]:
%%sql 
SHOW DATABASES;

In [ ]:
%%sql
SHOW TABLES in `bufstream`

In [1]:
%%sql
WITH category_totals AS (
    SELECT 
        line_item.product.category.name AS category,
        SUM(line_item.quantity * line_item.unit_price_cents) / 100.0 AS total_revenue_dollars,
        COUNT(DISTINCT val.invoice_id) AS num_invoices,
        SUM(line_item.quantity) AS total_units_sold
    FROM `bufstream`.`invoice`
    LATERAL VIEW EXPLODE(val.line_items) AS line_item
    GROUP BY category
),
grand_total AS (
    SELECT 
        'TOTAL' AS category,
        SUM(total_revenue_dollars) AS total_revenue_dollars,
        SUM(num_invoices) AS num_invoices,
        SUM(total_units_sold) AS total_units_sold
    FROM category_totals
)
SELECT * FROM category_totals
UNION ALL
SELECT * FROM grand_total
ORDER BY 
    CASE WHEN category = 'TOTAL' THEN 1 ELSE 0 END,
    total_revenue_dollars DESC;



25/10/21 19:50:34 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
                                                                                

category,total_revenue_dollars,num_invoices,total_units_sold
Electronics & Accessories,446101.980000,3804,15302
Home & Garden,408217.520000,3889,12548
Books & Stationery,393798.300000,3406,14670
Sports & Outdoors,362286.870000,3874,9313
Kitchen & Dining,288191.780000,3507,7622
Personal Care,193034.540000,3072,8346
TOTAL,2091630.990000,21552,67801
